## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Изучение общей информации

In [2]:
import pandas as pd
data = pd.read_csv('data.csv')

data.sample(2)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16615,1,NaN,33,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для сдачи
3704,0,-4288.71441,0,среднее,1,в разводе,3,F,сотрудник,0,191033.656419,автомобили


### Вывод

В таблице 21525 строк и 12 столбцов. Встречаются данные целочисленного, вещественного и object типов. Среди данных есть пропуски в значениях дохода и трудового стажа. Также присутствуют отрицательные значения в данных о трудовом стаже и детях. В графе об образовании значения записаны в разных регистрах (но это не критично, так как есть дополнительная информация об образовании, закодированная в числовом виде).

### Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
data = data.fillna(0)
data.isna().sum().sum()

0

### Вывод

По некоторым заемщикам нет информации об их трудоустройстве и доходах (всего 2174).
Эти значения были заполнены 0, так как равновероятно, что дохода и работы нет, как и то, что данные просто отсутствуют.

Я взяла для анализа второй вариант, так как с таким раскладом анализ будет более "строгим", чем если присваивать пустым значениям среднюю величину.

### Замена типа данных

In [5]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data['education'] = data['education'].str.lower() # приведем написание данных об образовании к нижнему регистру


### Вывод

Изменила тип данных float на int, чтоб было удобнее проводить анализ в дальнейшем методом astype('int')

### Обработка дубликатов

In [5]:
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

### Вывод

Для выявления дубликатов выбрала метод duplicated().sum(), чтоб определить общее количество повторяющихся строк.

Всего было выявлено 54 дубликата. Они могли появиться из-за того, что заемщик обращался в банк неоднократно, либо к разным сотрудникам банка. Таким образом данные о заемщике попали в базу более 1 раза.

Чтоб очистить данные от одинаковых строк (повторной информации о заемщике), воспользовалась методом drop_duplicated и перезапустила индексацию строк.

В конце проверила, чтоб сумма дубликатов в таблице = 0.

### Лемматизация

In [6]:
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
data['purpose'] += ' '  # добавила пробел к концу строки о цели кредита, чтоб избежать склейки слов
row = data['purpose'].sum() # сделала одну строку из всех назначений кредита для дальнейшего выявления лемм
lemmas = m.lemmatize(row) 
print(Counter(lemmas))

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, ' \n': 1})


### Вывод

Из всех выявленных значений лемм можно сделать вывод, что люди берут кредиты на несколько основных целей: покупка жилья/недвижимости, покупка автомобиля, свадьба, операция, образование, ремонт, сторительство, проведение мероприятия.

In [27]:
def age_group(age): # распределим заемщиков по возрастным критериям
    if age < 21:
        return 'ребенок'
    if 21 <= age < 65:
        return 'взрослый'
    if age >= 64:
        return 'пенсионер'
data['age_group'] = data['dob_years'].apply(age_group)

#-----------------------------------------------------------------------------------------------------------

def income_range(income): # распределим заемщиков по доходу
    if income  < 80000:
        return 'низкий'
    if 80000 <= income < 150000:
        return 'средний'
    if 150000 <= income < 200000:
        return 'высокий'
    if income >= 200000:
        return 'очень высокий'
data['income_range'] = data['total_income'].apply(income_range)

#-----------------------------------------------------------------------------------------------------------
    
def parents(kids): # распределим заемщиков по наличию детей
    if kids == 1:
        return '1 ребенок'
    if kids == 2:
        return '2 ребенка'
    if kids > 3:
        return 'многодетный'
    return 'бездетный'
data['parents'] = data['children'].apply(parents) 

#-----------------------------------------------------------------------------------------------------------

def family_keen(row): # распределим заемщиков по типам семейный/без семьи
    status = row['family_status']
    kids = row['parents']
   
    if status  == 'Не женат / не замужем':
        return 'без семьи'
    
    if status == 'в разводе':
        if kids == 'бездетный':
            return 'без семьи'
        return 'семейный'
    
    if status == 'вдовец / вдова':
        if kids == 'бездетный':
            return 'без семьи'
        return 'семейный' 
    
    if status == 'гражданский брак':
        if kids == 'бездетный':
            return 'без семьи'
        return 'семейный' 
    
    if status == 'женат / замужем':
        return 'семейный'
    
data['family_keen'] = data.apply(family_keen, axis=1)

#-----------------------------------------------------------------------------------------------------------
     
def purpose_category(category): # распрелелим заемщиков в зависимости от целей кредита
    if 'жи' in category:
        return 'жилье'
    if 'авто' in category:
        return 'авто'
    if 'свадьб' in category:
        return 'свадьба'
    if 'образов' in category:
        return 'образование'
    if 'операц' in category:
        return 'операция'
    if 'строит' in category:
        return 'строительство'
    return 'другое'
data['purpose_category'] = data['purpose'].apply(purpose_category) 



data.head(5)
    




,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,income_range,parents,family_keen,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,взрослый,очень высокий,1 ребенок,семейный,жилье
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,взрослый,средний,1 ребенок,семейный,авто
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,взрослый,средний,бездетный,семейный,жилье
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,взрослый,очень высокий,бездетный,семейный,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,взрослый,высокий,бездетный,без семьи,свадьба


### Вывод

Для упрощения дельнейшего анализа, я разделила всех заемщиков по разным категорям групп:

 1) по возрасту: на пенсионеров, взрослых и детей
 
 2) по уровню дохода: на людей с низким, средним, высоким и очень высоким доходом. Распределение было произведено в зависимости от примерного нахождения дохода в сравнении с квартилями: 80тыр - низкий доход; от 80 до 150 тыр - средний доход; от 150 до 200 тыр - высокий; больше 200 тыр - очень высокий
 
 3) по наличию детей: с 1 ребенком, с 2 детьми, многодетный (с 3 и более), без детей - бездетный
 
 4) по семейному положению: на семейных (сюда попали все, кто имеет супруга(у) и/или детей) и несемейных (сюда попали те, у кого нет супруга(и) и/или детей)


In [28]:
data['total_income'].describe().astype('int')

count      21454
mean      151010
std       109736
min            0
25%        89088
50%       135781
75%       195813
max      2265604
Name: total_income, dtype: int64

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [29]:
data.groupby('parents')[['debt']].mean().sort_values(by = 'debt', ascending = False) # подсчет доли наличия долгов по кредитам среди родителей и бездетных


,debt
parents,
многодетный,0.095238
2 ребенка,0.094542
1 ребенок,0.092346
бездетный,0.075408


### Вывод

Чем больше детей у заемщика, тем больше вероятность долгов по кредиту. Такое явление может быть связано с тем, что в любое время у заемщика могут появиться непредвиденные расходы на детей, что может помешать вовремя погасить платеж по кредиту.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [30]:
data.groupby('family_keen')[['debt']].mean().sort_values(by = 'family_keen')
# подсчет доли заемщицов с долгами по кредитам среди семейных и несемейных


,debt
family_keen,
без семьи,0.085435
семейный,0.078960


### Вывод

Несемейные люди более склонны иметь долги по кредитам

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [31]:
data.groupby('income_range')[['debt']].mean().sort_values(by ='debt', ascending = False)
# подсчет доли заемщиков с долгами по кредитам среди людей с разным доходом


,debt
income_range,
высокий,0.089364
средний,0.085034
низкий,0.078557
очень высокий,0.070667


### Вывод

Заемщики, чей доход составляет от 80 до 200 тыр имеют больше проблем с возвратом кредита в срок, тогда как люди с низким и очень высоким доходом, более чаще склонны отдавать кредит вовремя.
Возможно это связано с тем, что заемщики с низким доходом более ответственно относятся к тратам и возврату долгов, а заемщики с очень высоким доходом - просто имеют достаточно средств для погашения кредитов в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [32]:
data.groupby('purpose_category')[['debt']].mean().sort_values(by='debt', ascending = False)
# подсчет доли заемщиков с долгами по кредитам в зависимости от цели кредита


,debt
purpose_category,
авто,0.093590
образование,0.092200
свадьба,0.080034
жилье,0.072334


### Вывод

Больше всего долгов по кредитам в категории людей, берущих кредит на машину и образование. Но % невозврата в этих категорях лишь ненамного выше, чем в случаях кредита на свадьбу или жилье.

### Шаг 4. Общий вывод

В ходе исследования были выявлены следующие наблюдения:

1 - чем больше детей у заемщика, тем он более склонен к долгам по кредиту

2 - если у заемщика нет семьи, то заемщик с большей вероятностью будет иметь проблемы с погашением кредита

3 - заемщики со средним и высоким уровнем дохода имеют больше невозвратных кредитов, в отличии от тех, чей доход низкий или очень высокий

4 - по анализу целей кредита оказалось, что долгов больше у тех, кто брал деньги на покупку авто, либо на образование

Таким образом, самым благонадежным заемщиком банка является семейный человек без детей, либо с 1 ребенком, с низким или очень высоким уровнем дохода, берущий кредит на покупку жилья. Скорей всего, это будут молодые люди, кто недавно поженился, еще не успел обзавестись детьми и хочет приобрести собственное жилье.